# MIP4cluster

This is the source code for the paper Joint Problems in Learning Multiple Dynamical Systems.



# To start


## Gurobi

In [3]:
!pip install scikit-learn seaborn
from sklearn.metrics import f1_score
from scipy.io import arff
!pip install gurobipy
import gurobipy as gp
from gurobipy import *

In [4]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Clustering_EM_Heuristic/Joint-Xiaoyu/")

!export PYTHONPATH="$PYTHONPATH:/content/drive/MyDrive/Colab Notebooks/NCPOP-Colab Notebooks-data"
import os
os.environ['GRB_LICENSE_FILE'] = '/content/drive/MyDrive/gurobi.lic'  # Update this path with the actual license location

# import os
# os.environ['GRB_LICENSE_FILE'] = '/home/hexiaoyu/JointProblem/gurobi.lic'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import numpy as np
import pandas as pd
from copy import deepcopy
from pyomo.environ import *
import time
import sys
import os
import random

## Bonmin

In [6]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Clustering_EM_Heuristic/Joint-Xiaoyu/")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import sys
import os
import random

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

Getting files...
Done
-----------------------------------------------------------------
IDAES Extensions Build Versions
Solvers:  v3.4.2 20240811 ubuntu2204-x86_64
Library:  v3.4.2 20240811 ubuntu2204-x86_64



In [2]:
import numpy as np
import pandas as pd
from copy import deepcopy
from pyomo.environ import *
from sklearn.metrics import f1_score
from scipy.io import arff
import time

# Function

## MILP-IF-Gurobi

In [6]:
def ind_function(X, label, UB0, UB1, M, T):
    """Fit Estimator based on NCPOP Regressor model and predict y or produce residuals."""

    e = gp.Env()
    e.setParam('TimeLimit', 6*60)
    model = gp.Model(env=e)
    T -= 1
    I = len(X)

    # Var
    L = model.addVars(I, name="L", vtype='B')  # Indicator

    f0 = model.addVars(M, T, name="f0", vtype='C')
    phi0 = model.addVars(UB0, (T + 1), name="phi0", vtype='C')
    G0 = model.addVars(UB0, UB0, name="G0", vtype='C')
    F0 = model.addVars(M, UB0, name="F0", vtype='C')
    q0 = model.addVars(UB0, T, name="q0", vtype='C')
    p0 = model.addVars(M, T, name="p0", vtype='C')
    quatr0 = model.addVars(M, (T + 1), name="quatr0", vtype='C')
    quatr_hidden0 = model.addVars(UB0, T, name="quatr_hidden0", vtype='C')

    f1 = model.addVars(M, T, name="f1", vtype='C')
    phi1 = model.addVars(UB1, (T + 1), name="phi1", vtype='C')
    G1 = model.addVars(UB1, UB1, name="G1", vtype='C')
    F1 = model.addVars(M, UB1, name="F1", vtype='C')
    q1 = model.addVars(UB1, T, name="q1", vtype='C')
    p1 = model.addVars(M, T, name="p1", vtype='C')
    quatr1 = model.addVars(M, (T + 1), name="quatr1", vtype='C')
    quatr_hidden1 = model.addVars(UB1, T, name="quatr_hidden1", vtype='C')
    z0_squared = model.addVars(I, T, M, name="z0_squared", vtype='C')
    z1_squared = model.addVars(I, T, M, name="z1_squared", vtype='C')


    # Obj
    obj = gp.quicksum(
        (1 - L[i]) * z0_squared[i, t, m] + L[i] * z1_squared[i, t, m]
        for m in range(M) for t in range(T) for i in range(I)
    )
    obj += gp.quicksum(270 * q0[n0, t] ** 2 for n0 in range(UB0) for t in range(T))
    obj += gp.quicksum(270 * q1[n1, t] ** 2 for n1 in range(UB1) for t in range(T))
    obj += gp.quicksum(270 * p0[m, t] ** 2 + 270 * p1[m, t] ** 2 for m in range(M) for t in range(T))

    model.setObjective(obj, GRB.MINIMIZE)

    # Constraint
    model.addConstrs(z0_squared[i, t, m] == (X[i, t, m] - f0[m, t])**2 for i in range(I) for t in range(T) for m in range(M))
    model.addConstrs(z1_squared[i, t, m] == (X[i, t, m] - f1[m, t])**2 for i in range(I) for t in range(T) for m in range(M))

    for t in range(T):
        for m in range(M):
            model.addConstr(gp.quicksum(F0[m, n] * phi0[n, t+1] for n in range(UB0)) == quatr0[m, t+1])
            model.addConstr(gp.quicksum(F1[m, n] * phi1[n, t+1] for n in range(UB1)) == quatr1[m, t+1])
    model.addConstr(gp.quicksum(f0[m, t] - quatr0[m,(t+1)] - p0[m, t]for t in range(T) for m in range(M)) == 0)
    model.addConstr(gp.quicksum(f1[m, t] - quatr1[m,(t+1)] - p1[m, t]for t in range(T) for m in range(M)) == 0)
    for t in range(T):
        for n in range(UB0):
            model.addConstr(gp.quicksum(G0[n, nn] * phi0[nn, t] for nn in range(UB0)) == quatr_hidden0[n, t])
        for n in range(UB1):
            model.addConstr(gp.quicksum(G1[n, nn] * phi1[nn, t] for nn in range(UB1)) == quatr_hidden1[n, t])
    model.addConstr(gp.quicksum(phi0[n, t+1] - quatr_hidden0[n, t]- q0[n, t] for t in range(T) for n in range(UB0))  == 0)
    model.addConstr(gp.quicksum(phi1[n, t+1] - quatr_hidden1[n, t]- q1[n, t] for t in range(T) for n in range(UB1))  == 0)

    model.update()
    model.Params.NonConvex = 2

    model.optimize()

    if model.status == GRB.Status.OPTIMAL:
        print("THIS IS OPTIMAL SOLUTION")
    else:
        print("THIS IS NOT OPTIMAL SOLUTION")

    data_dict = {
        'G0': [model.getAttr("x", G0)[h, k] for (h, k) in model.getAttr("x", G0)],
        'F0': [model.getAttr("x", F0)[h, k] for (h, k) in model.getAttr("x", F0)],
        'G1': [model.getAttr("x", G1)[h, k] for (h, k) in model.getAttr("x", G1)],
        'F1': [model.getAttr("x", F1)[h, k] for (h, k) in model.getAttr("x", F1)]
    }

    label_out = np.array([round(model.getAttr("x", L)[h]) for h in model.getAttr("x", L)])

    # print(f'M={M}, T={T}')
    return model, label_out, data_dict


### IF-Test

In [7]:
def system_matrix(arr_G, arr_F, M, UB):
    threshold=0.001
    arr_rounded_G = np.round(arr_G, decimals=3)
    arr_rounded_F = np.round(arr_F, decimals=3)
    arr_rounded_G[np.abs(arr_rounded_G) < threshold] = 0
    arr_rounded_F[np.abs(arr_rounded_F) < threshold] = 0
    rows_G, cols_G = arr_G.shape
    rows_F, cols_F = arr_F.shape

    if not arr_rounded_G.any() and not arr_rounded_F.any():
        print("Warning: arr_rounded_G is empty or contains only zeros.")
        return arr_G, arr_F,0

    else:
        nonzero_rows_G, nonzero_cols_G = np.nonzero(arr_rounded_G.reshape(UB, UB))
        nonzero_rows_F, nonzero_cols_F = np.nonzero(arr_rounded_F.reshape(M,UB))
        def max_max(G):
            if G.size ==0:
                return 0
            else:
                return max(G)
        N = max(max_max(nonzero_rows_G), max_max(nonzero_cols_G), max_max(nonzero_cols_F)) + 1
        # print(f"N:{N}")
        system_G = arr_G[:N, :N]
        system_F = arr_F[:, :N]
        return system_G, system_F, N

def IF_test(X_data,X_label,UB0,UB1,name,win_size,seed):
  """
  X_data.shape: N,T,M, for ecg, X_data contains all the data
  X_lebel.shape: N     for both, X_label is like [1,1,1,0,0,0]
  name: 'lds' or 'ecg'
  """
  np.random.seed(seed)

  if name == 'lds':
    data_in = X_data
    T_idx = 3

  elif name == 'ecg':
    X_1 = X_data[X_data.target==b'1'].iloc[:,:-1].values
    # print(X_1.shape)
    X_2 = X_data[X_data.target==b'2'].iloc[:,:-1].values
    idx_1 = np.arange(len(X_1))
    idx_2 = np.arange(len(X_2))
    iidx_1 = random.sample(sorted(idx_1), 15)
    iidx_2 = random.sample(sorted(idx_2), 15)
    np.random.shuffle(iidx_1)
    np.random.shuffle(iidx_2)
    XX_1 = X_1[iidx_1]
    XX_2 = X_2[iidx_2]
    data_in = np.concatenate((XX_1, XX_2), axis=0)
    # print(data_in.shape)
    data_in = data_in.reshape(30,140,1)
    print(data_in.shape)
    T_idx = win_size

  I = np.random.randint(5, 16)  # data_in.shape[0]
  I_idx = np.random.choice(data_in.shape[0], size=I, replace=False)
  T = np.random.randint(30, 100)
  data_ = data_in[I_idx][:, T:(T+T_idx), :]
  label_in = X_label[I_idx]
  print("lds initialized label:", label_in)

  m = data_.shape[2]
  t = data_.shape[1]
  print("m,t:",m,t)
  model, label_out, data_dict = ind_function(X=data_,label=label_in, UB0=UB0,UB1=UB1,M=m,T=t)
  G0 = np.array(np.array(data_dict['G0']).reshape(UB0,UB0))
  F0 = np.array(np.array(data_dict['F0']).reshape(m,UB0))
  G1 = np.array(np.array(data_dict['G1']).reshape(UB1,UB1))
  F1 = np.array(np.array(data_dict['F1']).reshape(m,UB1))
  G0, F0, N0=system_matrix(G0, F0, m, UB=UB0)
  G1, F1, N1=system_matrix(G1, F1, m, UB=UB1)
  print(f"Hidden_State Dimension of System 1 is:{N0}.")
  print(f"Hidden_State Dimension of System 2 is:{N1}.")
  print(f"System matrices are G0: \n {G0} and \n F0:\n{F0}.")
  print(f"System matrices are G1: \n {G1} and \n F1:\n{F1}.")

  f1_1 = f1_score(label_in, label_out)
  f1_2 = f1_score(label_in, 1-label_out)

  if f1_1>f1_2:
      f1 = f1_1
  else:
      f1 = f1_2

  if np.sum(np.array(data_dict['G0']) == 0) >= (N0*N0+N0*m)*3/4 or np.sum(np.array(data_dict['F0']) == 0) >= (N0*N0+N0*m)*3/4 or np.sum(np.array(data_dict['G1']) == 0) >= (N1*N1+N1*m)*3/4 or np.sum(np.array(data_dict['F1']) == 0) >= (N1*N1+N1*m)*3/4 or f1==0:
     print("The System Matrices Is Sparse.")
     validation = 0
  else:
     print("The System Matrices Is Dense!")
     validation = 1
  print('f1-score:',f1)

  return f1,validation

## MILP-IF-Bonmin

In [3]:
#######################  CONCRETE MODEL  #######################

def obj_function(model):
  obj_1 = sum((1-model.L[i])*(model.X[i][t,m]-model.f[m,t])**2 + model.L[i]*(model.X[i][t,m]-model.ff[m,t])**2 for i in model.I for m in model.M for t in model.T1)
  obj_2 = 270*sum(model.p[m,t]**2 for m in model.M for t in model.T1) + 270*sum(model.q[n,t]**2 for n in model.N for t in model.T1)
  obj_3 = 270*sum(model.pp[m,t]**2 for m in model.M for t in model.T1) + 270*sum(model.qq[n,t]**2 for n in model.N for t in model.T1)
  return obj_1+obj_2+obj_3

def ind_function(X,label,N,M,T=20,option='bonmin'):

  I = len(X)
  # T = len(X[0])
  # print(I,T)

  #---------------------------------------------#
  # Dimension of G & F  - G:[N,N]  Fdash:[M,N]  #
  #---------------------------------------------#
  # N = 2
  # M = 1

  # Create model
  model = ConcreteModel()

  # Create index set
  model.N = RangeSet(0, N-1)
  model.M = RangeSet(0, M-1)
  model.I = RangeSet(0, I-1)
  model.T1 = RangeSet(0,T-1)
  model.T2 = RangeSet(0,T)

  # Create variables
  model.L = Var(model.I, within=NonNegativeIntegers, bounds=(0,1), initialize =label) #indicator

  model.f = Var(model.M, model.T1, domain=Reals)
  model.m = Var(model.N, model.T2, domain=Reals)
  model.G = Var(model.N, model.N, domain=Reals)
  model.Fdash = Var(model.M, model.N, domain=Reals)
  model.q = Var(model.N, model.T1, domain=Reals)
  model.p = Var(model.M, model.T1, domain=Reals)

  model.ff = Var(model.M, model.T1, domain=Reals)
  model.mm = Var(model.N, model.T2, domain=Reals)
  model.GG = Var(model.N, model.N, domain=Reals)
  model.FFdash = Var(model.M, model.N, domain=Reals)
  model.qq = Var(model.N, model.T1, domain=Reals)
  model.pp = Var(model.M, model.T1, domain=Reals)

  # Create parameters
  model.X = Param(model.I, initialize = X,mutable=False)

  # Create objective function
  model.OBJ = Objective(rule=obj_function, sense=minimize)

  # Create constrains
  model.con1 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) >=0)
  model.con2 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) <=0)
  model.con3 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) >=0)
  model.con4 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) <=0)

  model.con5 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) >=0)
  model.con6 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) <=0)
  model.con7 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) >=0)
  model.con8 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) <=0)

  solver = SolverFactory(option)
  solver.solve(model, tee=True)
  # solver.solve(model, tee=True).write()

  # solver = SolverManagerFactory('neos')
  # result = solver.solve(model, opt='mosek')

  # model.display()
  iid = []
  for i in model.L:
    iid.append(model.L[i].value)

  return model, np.array(iid)

In [18]:
def IF_test(X_data,X_label,n,name,win_size,seed):
  """
  X_data.shape: N,T,M, for ecg, X_data contains all the data
  X_lebel.shape: N     for both, X_label is like [1,1,1,0,0,0]
  name: 'lds' or 'ecg'
  """
  np.random.seed(seed)

  if name == 'lds':
    data_in = X_data

  elif name == 'ecg':
    X_1 = X_data[X_data.target==b'1'].iloc[:,:-1].values
    X_2 = X_data[X_data.target==b'2'].iloc[:,:-1].values
    idx_1 = np.arange(len(X_1))
    idx_2 = np.arange(len(X_2))
    iidx_1 = random.sample(sorted(idx_1), 15)
    iidx_2 = random.sample(sorted(idx_2), 15)
    np.random.shuffle(iidx_1)
    np.random.shuffle(iidx_2)
    XX_1 = X_1[iidx_1]
    XX_2 = X_2[iidx_2]
    data_in = np.concatenate((XX_1, XX_2), axis=0).reshape(30,140,1)

    if win_size == 140:
      data_in = data_in
    else:
      s = random.sample(sorted(np.arange(140-win_size)),1)
      s = s[0]
      data_in = data_in[:,s:(s+win_size),:]
    print(data_in.shape)

  idx = np.arange(len(data_in))
  np.random.shuffle(idx)
  # print("initialized index:", idx)
  label_in = X_label[idx]
  print("lds initialized label:", label_in)

  m = data_in.shape[2]
  t = data_in.shape[1]
  print("m,t:",m,t)
  model, label_out = ind_function(X=data_in, label=label_in, N=n,M=m,T=t)
  f1_1 = f1_score(X_label, label_out)
  f1_2 = f1_score(X_label, 1-label_out)

  if f1_1>f1_2:
      f1 = f1_1
  else:
      f1 = f1_2
  print('f1:',f1)

  return f1

## EM Heuristic

In [27]:
#######################  CONCRETE MODEL  #######################

def obj_function_2(model):
  obj_1 = sum((model.K[k][t,m]-model.f[m,t])**2 for k in model.KK for m in model.M for t in model.T1)
  obj_2 = sum((model.L[l][t][m]-model.ff[m,t])**2 for l in model.LL for m in model.M for t in model.T1)
  obj_3 = 270*sum(model.p[m,t]**2 for m in model.M for t in model.T1) + 270*sum(model.q[n,t]**2 for n in model.N for t in model.T1)
  obj_4 = 270*sum(model.pp[m,t]**2 for m in model.M for t in model.T1) + 270*sum(model.qq[n,t]**2 for n in model.N for t in model.T1)
  return obj_1+obj_2+obj_3+obj_4

def SimCom(K,L,T,M,N,MTS=True,option='bonmin'): # Dimension of G & F   # G:[N,N]  F_dash:[M,N]
  """option: 'bonmin', 'cplex', 'mosek'..."""

  KK = len(K)
  LL = len(L)
  # T = 5
  print('cluster length:',KK,LL)

  # Create model
  model = ConcreteModel()
  # # Dimension of G & F   # G:[N,N]  F_dash:[M,N]
  # M = 2
  # N = 2
  # Create index set
  model.M = RangeSet(0, M-1)
  model.N = RangeSet(0, N-1)
  model.KK = RangeSet(0,KK-1)
  model.LL = RangeSet(0,LL-1)
  model.T1 = RangeSet(0,T-1)
  model.T2 = RangeSet(0,T)

  # Create variables
  model.f = Var(model.M, model.T1, domain=Reals)
  model.m = Var(model.N, model.T2, domain=Reals)
  model.G = Var(model.N, model.N, domain=Reals)
  model.Fdash = Var(model.M, model.N, domain=Reals)
  model.q = Var(model.N, model.T1, domain=Reals)
  model.p = Var(model.M, model.T1, domain=Reals)

  model.ff = Var(model.M, model.T1, domain=Reals)
  model.mm = Var(model.N, model.T2, domain=Reals)
  model.GG = Var(model.N, model.N, domain=Reals)
  model.FFdash = Var(model.M, model.N, domain=Reals)
  model.qq = Var(model.N, model.T1, domain=Reals)
  model.pp = Var(model.M, model.T1, domain=Reals)

  # Create parameters
  model.K = Param(model.KK,initialize = K,mutable=False)
  model.L = Param(model.LL,initialize = L,mutable=False)

  # Create objective function

  model.OBJ = Objective(rule=obj_function_2, sense=minimize)

  # Create constrains
  model.con1 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) >=0)
  model.con2 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) <=0)
  model.con3 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) >=0)
  model.con4 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) <=0)

  model.con5 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) >=0)
  model.con6 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) <=0)
  model.con7 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) >=0)
  model.con8 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) <=0)

  solver = SolverFactory(option)
  solver.solve(model)
  # solver.solve(model, tee=True).write()

  # solver = SolverManagerFactory('neos')
  # result = solver.solve(model, opt='mosek')


  # model.pprint()
  return model

In [28]:
#######################  CONCRETE MODEL  #######################

# def obj_function_2(model):
#   obj_1 = sum((model.K[k][t,m]-model.f[m,t])**2 for k in model.KK for m in model.M for t in model.T1)
#   obj_2 = sum((model.L[l][t][m]-model.ff[m,t])**2 for l in model.LL for m in model.M for t in model.T1)
#   obj_3 = 0.5*sum(model.p[m,t]**2 for m in model.M for t in model.T1) + 0.5*sum(model.q[n,t]**2 for n in model.N for t in model.T1)
#   obj_4 = 0.5*sum(model.pp[m,t]**2 for m in model.M for t in model.T1) + 0.5*sum(model.qq[n,t]**2 for n in model.N for t in model.T1)
#   return obj_1+obj_2+obj_3+obj_4

# model with G constrains
def SimCom_norm(K,L,T,M,N,MTS=True,option='bonmin'): # Dimension of G & F   # G:[N,N]  F_dash:[M,N]
  KK = len(K)
  LL = len(L)
  # T = 5
  print('cluster length:',KK,LL)

  # Create model
  model = ConcreteModel()
  # # Dimension of G & F   # G:[N,N]  F_dash:[M,N]
  # M = 2
  # N = 2
  # Create index set
  model.M = RangeSet(0, M-1)
  model.N = RangeSet(0, N-1)
  model.KK = RangeSet(0,KK-1)
  model.LL = RangeSet(0,LL-1)
  model.T1 = RangeSet(0,T-1)
  model.T2 = RangeSet(0,T)

  # Create variables
  model.f = Var(model.M, model.T1, domain=Reals)
  model.m = Var(model.N, model.T2, domain=Reals)
  model.G = Var(model.N, model.N, domain=Reals)
  model.Fdash = Var(model.M, model.N, domain=Reals)
  model.q = Var(model.N, model.T1, domain=Reals)
  model.p = Var(model.M, model.T1, domain=Reals)

  model.ff = Var(model.M, model.T1, domain=Reals)
  model.mm = Var(model.N, model.T2, domain=Reals)
  model.GG = Var(model.N, model.N, domain=Reals)
  model.FFdash = Var(model.M, model.N, domain=Reals)
  model.qq = Var(model.N, model.T1, domain=Reals)
  model.pp = Var(model.M, model.T1, domain=Reals)

  # Create parameters
  model.K = Param(model.KK,initialize = K,mutable=False)
  model.L = Param(model.LL,initialize = L,mutable=False)

  # Create objective function

  model.OBJ = Objective(rule=obj_function_2, sense=minimize)

  # Create constrains
  model.con1 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) >=0)
  model.con2 = Constraint(expr=sum(model.f[m,t]-sum(model.Fdash[m,n]*model.m[n,t+1] for n in model.N) -model.p[m,t] for m in model.M for t in model.T1) <=0)
  model.con3 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) >=0)
  model.con4 = Constraint(expr=sum(model.m[n,t+1]-sum(model.G[n,n2]*model.m[n2,t] for n2 in model.N) - model.q[n,t] for n in model.N for t in model.T1) <=0)

  model.con5 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) >=0)
  model.con6 = Constraint(expr=sum(model.ff[m,t]-sum(model.FFdash[m,n]*model.mm[n,t+1] for n in model.N) -model.pp[m,t] for m in model.M for t in model.T1) <=0)
  model.con7 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) >=0)
  model.con8 = Constraint(expr=sum(model.mm[n,t+1]-sum(model.GG[n,n2]*model.mm[n2,t] for n2 in model.N) - model.qq[n,t] for n in model.N for t in model.T1) <=0)

  # G for normalization
  model.con9 = Constraint(expr=sum(model.G[i,j] for i in model.N for j in model.N)-1==0)
  model.con10 = Constraint(expr=sum(model.GG[i,j] for i in model.N for j in model.N)-1==0)

  # model.con9 = Constraint(expr=sum(model.m[i,j] for i in model.N for j in model.T2)-1==0)
  # model.con10 = Constraint(expr=sum(model.mm[i,j] for i in model.N for j in model.T2)-1==0)

  solver = SolverFactory(option)
  solver.solve(model)


  # model.pprint()
  return model

In [29]:
def EM_process(X_data, label, name, T, N, M, seed, MTS=True,option='bonmin',norm=True):
  """ T:win_size, N:dim_G, M:dim_F """

  np.random.seed(seed)

  if name == 'lds':
    X = X_data

  elif name == 'ecg':
    X_1 = X_data[X_data.target==b'1'].iloc[:,:-1].values
    X_2 = X_data[X_data.target==b'2'].iloc[:,:-1].values
    idx_1 = np.arange(len(X_1))
    idx_2 = np.arange(len(X_2))
    iidx_1 = random.sample(sorted(idx_1), 15)
    iidx_2 = random.sample(sorted(idx_2), 15)
    np.random.shuffle(iidx_1)
    np.random.shuffle(iidx_2)
    XX_1 = X_1[iidx_1]
    XX_2 = X_2[iidx_2]
    X = np.concatenate((XX_1, XX_2), axis=0).reshape(30,140,1)

    win_size = T
    if win_size == 140:
      X = X
    else:
      s = random.sample(sorted(np.arange(140-win_size)),1)[0]
      X = X[:,s:(s+win_size),:]
      print(s,X.shape)

  ### cluster initialization ###
  np.random.seed(seed)

  idx = np.random.randint(0,2,len(X))
  print(idx)
  a = np.where(idx==0)[0]
  b = np.where(idx==1)[0]
  print(a,b)
  K = [X[i] for i in a]  #idx = 0
  L = [X[i] for i in b]  #idx = 1
  print(len(K),len(L),K[0].shape)

  ### EM ###
  for i in range(10):
    idx_= deepcopy(idx)
    print(i, "start: idx_=", idx_)

    if norm == True:
      model = SimCom_norm(K,L,T=T,M=M, N=N,MTS=MTS,option=option)

    model = SimCom(K,L,T=T,M=M, N=N,MTS=MTS,option=option)
    pred_k = model.f # index obj, model.f[i].value
    pred_l = model.ff

    for n in range(len(X)):
        cost_k = sum((X[n][i,m]-pred_k[m,i].value)**2 for m in range(M) for i in range(T))
        cost_l = sum((X[n][i,m]-pred_l[m,i].value)**2 for m in range(M) for i in range(T))
        # print(cost_k, cost_l)
        if cost_k < cost_l: # K cluster --> 0
            idx[n] = 0
        else:
            idx[n] = 1

    print(i, "changed: idx=", idx)
    if np.array_equal(idx_,idx):
        print(idx_, "end")
        break
    else:
        idx_ = deepcopy(idx)
        a = np.where(idx==0)[0]
        b = np.where(idx==1)[0]
        K = [X[i] for i in a]  #idx = 0
        L = [X[i] for i in b]  #idx = 1
        print('K:',len(K),'\n','L:',len(L))

  f1_1 = f1_score(label, idx_)
  f1_2 = f1_score(label, 1-idx_)

  if f1_1>f1_2:
      f1 = f1_1
  else:
      f1 = f1_2
  print('f1:',f1)

  return f1

# Synthetic Data Test

## Data Loader

In [7]:
path_list = ['./data/2_2_test.npy', './data/3_2_test.npy', './data/4_2_test.npy']
data_list = []
label_list = []
for path in path_list:
    data = np.load(path)
    T = 20
    M = 2
    X_data = data.reshape(-1,T,M)
    X_label = np.concatenate((np.zeros(16),np.ones(16)),axis=0)
    print(X_data.shape, X_label)
    data_list.append(X_data)
    label_list.append(X_label)

(320, 20, 2) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
(320, 20, 2) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
(320, 20, 2) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]


## MILP-IF-Gurobi Test

In [ ]:
N=10
I=16
T=20
f1_if_Gurobi_lds = []  # 5*10*3
duration_if_Gurobi_lds = []
for n,data in enumerate(data_list):
    X_1 = data[:N*I].reshape(N,I,20,2)[:,:,:T,:]
    X_2 = data[(320-N*I):].reshape(N,I,20,2)[:,:,:T,:]

    t_start = time.time()

    for i in range(N):
        xx_1 = X_1[i]
        xx_2 = X_2[i]
        xx_all = np.concatenate((xx_1,xx_2),axis=0)
        label = np.concatenate((np.zeros(I),np.ones(I)),axis=0)
        for j in range(5):
            f1,validation = IF_test(xx_all,label,5,5,'lds',20,(30+i+j))
            print(f"n:{n+2}. regularization:{reg}.F1:{f1}")
            f1_if_Gurobi_lds.append(f1)
    duration_if_Gurobi_lds.append(time.time()-t_start)

Streaming output truncated to the last 5000 lines.
H 2255  1976                       0.1061882    0.00000   100%   4.5    4s
H 2385  2106                       0.1061881    0.00000   100%   4.4    4s
  2540  2263    0.08574 1140    4    0.10619    0.00000   100%   4.4    5s
H 2872  2372                       0.1061878    0.00000   100%   4.4    6s
H 2898  2282                       0.1061872    0.00000   100%   4.4    6s
H 5862  4155                       0.1061869    0.00000   100%   2.4    8s
H 6876  4733                       0.1061869    0.00000   100%   2.1    9s
  7394  5080    0.06965 2438    5    0.10619    0.00000   100%   2.0   10s
H 7760  5229                       0.1061865    0.00000   100%   1.9   10s
H 9788  7228                       0.1061864    0.00000   100%   1.6   11s
H11816  9256                       0.1061852    0.00000   100%   1.5   13s
H12232  9672                       0.1058949    0.00000   100%   1.4   13s
H12466  9906                       0.1058928    0

## MILP-IF Test

In [ ]:
f1_list_if_lds = []  # 5*10*3
duration_if_lds = []
for n,data in enumerate(data_list):
    X_1 = data[:160].reshape(10,16,20,2)
    X_2 = data[160:].reshape(10,16,20,2)

    t_start = time.time()

    for i in range(10):
      xx_1 = X_1[i]
      xx_2 = X_2[i]
      xx_all = np.concatenate((xx_1,xx_2),axis=0)
      label = np.concatenate((np.zeros(16),np.ones(16)),axis=0)

      for j in range(5):
        print("n:",n+2)
        f1 = IF_test(xx_all,label,n+2,'lds',20,(30+i+j))
        print(f1)
        f1_list_if_lds.append(f1)
    duration_if_lds.append(time.time()-t_start)

Streaming output truncated to the last 5000 lines.
m,t: 2 20
Bonmin 1.8.8 using Cbc 2.10.10 and Ipopt 3.13.2
bonmin: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:

## EM Test

In [ ]:
f1_list_em_lds = []  # 5*10*3
# f1_test = np.zeros([3,50])
duration_em_lds = []

for n,data in enumerate(data_list):
    X_1 = data[:160].reshape(10,16,20,2)
    X_2 = data[160:].reshape(10,16,20,2)

    t_start = time.time()

    for i in range(10):
      xx_1 = X_1[i]
      xx_2 = X_2[i]
      xx_all = np.concatenate((xx_1,xx_2),axis=0)
      label = np.concatenate((np.zeros(16),np.ones(16)),axis=0)

      for j in range(5):
        f1 = EM_process(xx_all, label,'lds', 20, (n+2), 2, (30+i+j), MTS=True)
        # f1_test[n,(i*5+j)] = f1
        f1_list_em_lds.append(f1)
    duration_em_lds.append(time.time()-t_start)

[1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0]
[ 4  5  7 11 21 22 23 24 25 26 27 28 29 31] [ 0  1  2  3  6  8  9 10 12 13 14 15 16 17 18 19 20 30]
14 18 (20, 2)
0 start: idx_= [1 1 1 1 0 0 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0]
cluster length: 14 18
0 changed: idx= [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
K: 16 
 L: 16
1 start: idx_= [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
cluster length: 16 16
1 changed: idx= [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] end
f1: 1.0
[0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 0 0 0 1 1]
[ 0  2  3  4  6  7  8 10 11 12 13 14 15 16 17 18 23 25 27 28 29] [ 1  5  9 19 20 21 22 24 26 30 31]
21 11 (20, 2)
0 start: idx_= [0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 0 0 0 1 1]
cluster length: 21 11
0 changed: idx= [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 

## Result

In [ ]:
f1_if_Gurobi_lds = np.array(f1_if_Gurobi_lds).reshape(3,50)  #3*5 第二次实验结果
f1_if_Gurobi_lds_mean = np.mean(f1_if_Gurobi_lds, axis=1)
f1_if_Gurobi_lds_std = np.std(f1_if_Gurobi_lds, axis=1)
f1_if_Gurobi_lds_mean, f1_if_Gurobi_lds_std, duration_if_Gurobi_lds

(array([0.80218765, 0.91920282, 0.8106027 ]),
 array([0.17744058, 0.14849509, 0.17163057]),
 [2443.530668735504, 2271.3195612430573, 2693.564868211746])

In [ ]:
f1_if_lds = np.array(f1_list_if_lds).reshape(3,50)  #3*5 第二次实验结果
f1_if_lds_mean = np.mean(f1_if_lds, axis=1)
f1_if_lds_std = np.std(f1_if_lds, axis=1)
f1_if_lds_mean, f1_if_lds_std, duration_if_lds

(array([0.80675238, 0.80254963, 0.71141706]),
 array([0.1285283 , 0.20235134, 0.12644587]),
 [45.037455320358276, 53.110230684280396, 51.55267930030823])

In [ ]:
f1_em_lds = np.array(f1_list_em_lds).reshape(3,50)  #3*5 第二次实验结果
f1_em_lds_mean = np.mean(f1_em_lds, axis=1)
f1_em_lds_std = np.std(f1_em_lds, axis=1)
f1_em_lds_mean, f1_em_lds_std, duration_em_lds

(array([0.78238274, 0.78804724, 0.71839602]),
 array([0.14714519, 0.19711908, 0.13534359]),
 [55.8861517906189, 63.27428460121155, 53.863877296447754])

In [ ]:
# for value in [f1_if_Gurobi_lds_mean,f1_if_Gurobi_lds_std, duration_if_Gurobi_lds, f1_em_lds_mean, f1_em_lds_std, duration_em_lds, f1_if_lds_mean, f1_if_lds_std, duration_if_lds]:
with open('f1_if_Gurobi_lds_mean.npy', 'wb') as f:
    np.save(f, f1_if_Gurobi_lds_mean)
with open('f1_if_Gurobi_lds_std.npy', 'wb') as f:
    np.save(f, f1_if_Gurobi_lds_std)
with open('duration_if_Gurobi_lds.npy', 'wb') as f:
    np.save(f, duration_if_Gurobi_lds)
with open('f1_em_lds_mean.npy', 'wb') as f:
    np.save(f, f1_em_lds_mean)
with open('f1_em_lds_std.npy', 'wb') as f:
    np.save(f, f1_em_lds_std)
with open('duration_em_lds.npy', 'wb') as f:
    np.save(f, duration_em_lds)
with open('f1_if_lds_mean.npy', 'wb') as f:
    np.save(f, f1_if_lds_mean)
with open('f1_if_lds_std.npy', 'wb') as f:
    np.save(f, f1_if_lds_std)
with open('duration_if_lds.npy', 'wb') as f:
    np.save(f, duration_if_lds)

# ECG Data Test

## Data Loader

In [8]:
def a2p(path):
    # load data
    data, meta = arff.loadarff(path)

    # turn to DataFrame
    df = pd.DataFrame(data)

    return df

In [9]:
trainpath = "./data/ECG5000_TRAIN.arff"
# trainpath = "./dataset/ECG5000_TRAIN.arff"

In [10]:
train = a2p(trainpath)
print(train.target.value_counts())
train.head()

target
b'1'    292
b'2'    177
b'4'     19
b'3'     10
b'5'      2
Name: count, dtype: int64


,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att132,att133,att134,att135,att136,att137,att138,att139,att140,target
0,-0.112522,-2.827204,-3.773897,-4.349751,-4.376041,-3.474986,-2.181408,-1.818286,-1.250522,-0.477492,...,0.792168,0.933541,0.796958,0.578621,0.257740,0.228077,0.123431,0.925286,0.193137,b'1'
1,-1.100878,-3.996840,-4.285843,-4.506579,-4.022377,-3.234368,-1.566126,-0.992258,-0.754680,0.042321,...,0.538356,0.656881,0.787490,0.724046,0.555784,0.476333,0.773820,1.119621,-1.436250,b'1'
2,-0.567088,-2.593450,-3.874230,-4.584095,-4.187449,-3.151462,-1.742940,-1.490659,-1.183580,-0.394229,...,0.886073,0.531452,0.311377,-0.021919,-0.713683,-0.532197,0.321097,0.904227,-0.421797,b'1'
3,0.490473,-1.914407,-3.616364,-4.318823,-4.268016,-3.881110,-2.993280,-1.671131,-1.333884,-0.965629,...,0.350816,0.499111,0.600345,0.842069,0.952074,0.990133,1.086798,1.403011,-0.383564,b'1'
4,0.800232,-0.874252,-2.384761,-3.973292,-4.338224,-3.802422,-2.534510,-1.783423,-1.594450,-0.753199,...,1.148884,0.958434,1.059025,1.371682,1.277392,0.960304,0.971020,1.614392,1.421456,b'1'


## MILP-IF-Gurobi Test

In [11]:
f1_list_if_Gurobi_ecg = []  #50*3(n)*5(t)
duration_if_Gurobi_ecg = []
validation_if_Gurobi_ecg = []
label = np.concatenate((np.zeros(15),np.ones(15)),axis=0)

for t in [2,3,4,5,6]:
  for n in [2,3,4]:
    t_start = time.time()
    for i in range(50):
      f1,validation = IF_test(train,label,n,n,'ecg',t,i//5+35)
      f1_list_if_Gurobi_ecg.append(f1)
      validation_if_Gurobi_ecg.append(validation)
    duration_if_Gurobi_ecg.append(time.time()-t_start)

Streaming output truncated to the last 5000 lines.
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Non-default parameters:
TimeLimit  360
NonConvex  2

Academic license 2611849 - for non-commercial use only - registered to he___@cvut.cz
Optimize a model with 4 rows, 321 columns and 150 nonzeros
Model fingerprint: 0x6e2af497
Model has 160 quadratic objective terms
Model has 160 quadratic constraints
Variable types: 310 continuous, 11 integer (11 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e-01, 2e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
  QRHS range       [5e-03, 9e-01]
Presolve added 100 rows and 0 columns
Presolve removed 0 rows and 2 columns
Presolve time: 0.01s
Presolved: 1205 rows, 901 columns, 2871 nonzeros
Presolved model has 220 SOS constraint(s)
Presolved model has 50 quadrat

## MILP-IF Test

In [21]:
f1_list_if_ecg_cd = []  #50*3(n)*5(t)
duration_if_ecg_cd = []
label = np.concatenate((np.zeros(15),np.ones(15)),axis=0)

for t in [2,3,4,5,6]:
  for n in [2,3,4]:
    t_start = time.time()
    for i in range(50):
      f1 = IF_test(train,label,n,'ecg',t,i//5+35)
      f1_list_if_ecg_cd.append(f1)
    duration_if_ecg_cd.append(time.time()-t_start)


Streaming output truncated to the last 5000 lines.
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://www.hsl.rl.ac.uk.
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0

In [ ]:
f1_list_if_ecg = []  #50*3(n)*5(t)
duration_if_ecg = []
label = np.concatenate((np.zeros(15),np.ones(15)),axis=0)

for t in [30,60,90,120,140]:
  for n in [2,3,4]:
    t_start = time.time()
    for i in range(50):
      f1 = IF_test(train,label,n,'ecg',t,i//5+35)
      f1_list_if_ecg.append(f1)
    duration_if_ecg.append(time.time()-t_start)


Streaming output truncated to the last 5000 lines.
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://www.hsl.rl.ac.uk.
******************************************************************************

NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 772.24506       45 1.275665
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 772.2451        2 0.131487
NLP0012I 
              Num      Status      Obj             It       time                 Location
NLP0014I             1         OPT 772.2451        2 0.132823
Cbc0012I Integer solution of 772.2451 found by DiveMIPFractional after 0 iterations and 0 nodes (0.27 seconds)
NLP0014I             2         OPT 772.2451        2 0.136592
NLP0014I             3         OPT 772.2451   

## EM Test

In [31]:
f1_list_em_ecg_cd = []  #50*3(n)*5(t)
duration_em_ecg_cd = []
label = np.concatenate((np.zeros(15),np.ones(15)),axis=0)

for t in [2,3,4,5,6]:
  for n in [2,3,4]:
    t_start = time.time()
    for i in range(50):
      f1 = EM_process(X_data=train, label=label,name='ecg', T=t, N=n, M=1, seed=i, MTS=True, norm=False)
      f1_list_em_ecg_cd.append(f1)
    duration_em_ecg_cd.append(time.time()-t_start)


Streaming output truncated to the last 5000 lines.
 L: 16
1 start: idx_= [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
cluster length: 14 16
1 changed: idx= [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0] end
f1: 0.9655172413793104
52 (30, 5, 1)
[0 1 0 0 0 1 1 1 0 1 1 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0]
[ 0  2  3  4  8 11 12 13 19 20 22 24 25 26 27 28 29] [ 1  5  6  7  9 10 14 15 16 17 18 21 23]
17 13 (5, 1)
0 start: idx_= [0 1 0 0 0 1 1 1 0 1 1 0 0 0 1 1 1 1 1 0 0 1 0 1 0 0 0 0 0 0]
cluster length: 17 13
0 changed: idx= [0 0 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 1 0 0]
K: 16 
 L: 14
1 start: idx_= [0 0 0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 1 0 0]
cluster length: 16 14
1 changed: idx= [0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 0]
K: 12 
 L: 18
2 start: idx_= [0 1 1 1 1 1 0 0 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 0 0 0 1 0 0]
cluster length: 12 18
2 changed: i

In [ ]:
f1_list_em_ecg = []  #50*3(n)*5(t)
label = np.concatenate((np.zeros(15),np.ones(15)),axis=0)

duration_em_ecg = [] #3*5

for t in [30,60,90,120,140]:
  for n in [2,3,4]:
    t_start = time.time()
    for i in range(50):
      f1 = EM_process(X_data=train, label=label,name='ecg', T=t, N=n, M=1, seed=i, MTS=True, norm=False)
      f1_list_em_ecg.append(f1)
    duration_em_ecg.append(time.time()-t_start)


## Result

In [3]:
f1_if_Gurobi_ecg = np.array(f1_list_Gurobi_if_ecg_cd).reshape(5,3,50)
f1_if_Gurobi_ecg_mean = np.mean(f1_if_Gurobi_ecg, axis=2)
f1_if_Gurobi_ecg_std = np.std(f1_if_Gurobi_ecg, axis=2)
f1_if_Gurobi_ecg_mean, f1_if_Gurobi_ecg_std

(array([[0.6       , 0.60358974, 0.61858974],
        [0.63399123, 0.66666667, 0.66666667],
        [0.68581699, 0.72173616, 0.74912281],
        [0.76022589, 0.7960114 , 0.8       ],
        [0.83259316, 0.85892857, 0.89992877]]),
 array([[0.13298464, 0.12938451, 0.14816592],
        [0.13948918, 0.14489676, 0.14688035],
        [0.13076946, 0.1266246 , 0.15974359],
        [0.11725936, 0.13067604, 0.14159441],
        [0.11694632, 0.09198769, 0.10993197]]))

In [ ]:
f1_em_ecg_cd = np.array(f1_list_em_ecg_cd).reshape(5,3,50)
f1_em_ecg_mean_cd = np.mean(f1_em_ecg_cd, axis=2)
f1_em_ecg_std_cd = np.std(f1_em_ecg_cd, axis=2)
f1_em_ecg_mean_cd, f1_em_ecg_std_cd

In [ ]:
f1_if_ecg_cd = np.array(f1_list_if_ecg_cd).reshape(5,3,50)
f1_if_ecg_mean_cd = np.mean(f1_if_ecg_cd, axis=2)
f1_if_ecg_std_cd = np.std(f1_if_ecg_cd, axis=2)
f1_if_ecg_mean_cd, f1_if_ecg_std_cd

In [ ]:
duration_if_Gurobi_ecg, duration_em_ecg_cd,duration_if_ecg_cd

In [ ]:
f1_em_ecg = np.array(f1_list_em_ecg).reshape(5,3,50)
f1_em_ecg_mean = np.mean(f1_em_ecg, axis=2)
f1_em_ecg_std = np.std(f1_em_ecg, axis=2)
f1_em_ecg_mean, f1_em_ecg_std

In [ ]:
f1_if_ecg = np.array(f1_list_if_ecg).reshape(5,3,50)
f1_if_ecg_mean = np.mean(f1_if_ecg, axis=2)
f1_if_ecg_std = np.std(f1_if_ecg, axis=2)
f1_if_ecg_mean, f1_if_ecg_std

In [ ]:
duration_em_ecg,duration_if_ecg

In [37]:
# for value in [f1_if_Gurobi_ecg_mean, f1_if_Gurobi_ecg_std,duration_if_Gurobi_ecg, f1_em_ecg_mean, f1_em_ecg_std, duration_em_ecg, f1_if_ecg_mean, f1_if_ecg_std, duration_if_ecg]:
with open('./Result-Real/f1_if_Gurobi_ecg_mean.npy', 'wb') as f:
    np.save(f, f1_if_Gurobi_ecg_mean)
with open('./Result-Real/f1_if_Gurobi_ecg_std.npy', 'wb') as f:
    np.save(f, f1_if_Gurobi_ecg_std)
with open('./Result-Real/duration_if_Gurobi_ecg.npy', 'wb') as f:
    np.save(f, duration_if_Gurobi_ecg)
with open('./Result-Real/f1_em_ecg_mean.npy', 'wb') as f:
    np.save(f, f1_em_ecg_mean)
with open('./Result-Real/f1_em_ecg_std.npy', 'wb') as f:
    np.save(f, f1_em_ecg_std)
with open('./Result-Real/duration_em_ecg.npy', 'wb') as f:
    np.save(f, duration_em_ecg)
with open('./Result-Real/f1_if_ecg_mean.npy', 'wb') as f:
    np.save(f, f1_if_ecg_mean)
with open('./Result-Real/f1_if_ecg_std.npy', 'wb') as f:
    np.save(f, f1_if_ecg_std)
with open('./Result-Real/duration_if_ecg.npy', 'wb') as f:
    np.save(f, duration_if_ecg)

with open('./Result-Real/cd/f1_em_ecg_mean_cd.npy', 'wb') as f:
    np.save(f, f1_em_ecg_mean_cd)
with open('./Result-Real/cd/f1_em_ecg_std_cd.npy', 'wb') as f:
    np.save(f, f1_em_ecg_std_cd)
with open('./Result-Real/cdduration_em_ecg|_cd.npy', 'wb') as f:
    np.save(f, duration_em_ecg_cd)
with open('./Result-Real/cd/f1_if_ecg_mean_cd.npy', 'wb') as f:
    np.save(f, f1_if_ecg_mean_cd)
with open('./Result-Real/cd/f1_if_ecg_std_cd.npy', 'wb') as f:
    np.save(f, f1_if_ecg_std_cd)
with open('./Result-Real/cd/duration_if_ecg_cd.npy', 'wb') as f:
    np.save(f, duration_if_ecg_cd)

# ECG Data Test for Comparison with NCPOP

In [ ]:
f1_list_em_ecg_2 = []  #50*3(n)*5(t)
label = np.concatenate((np.zeros(5),np.ones(5)),axis=0)

duration_em_ecg_2 = [] #3*5

for t in [10,20,30]:
  for n in [2,3,4]:
    t_start = time.time()
    for i in range(5):
      f1 = EM_process(X_data=train, label=label,name='ecg', T=t, N=n, M=1, seed=i, MTS=True, norm=False)
      f1_list_em_ecg_2.append(f1)
    duration_em_ecg_2.append(time.time()-t_start)

In [ ]:
f1_em_ecg_2 = np.array(f1_list_em_ecg_2).reshape(3,3,5)  #3*5 第二次实验结果
f1_em_ecg_2_mean = np.mean(f1_em_ecg_2, axis=2)
f1_em_ecg_2_std = np.std(f1_em_ecg_2, axis=2)
f1_em_ecg_2_mean, f1_em_ecg_2_std

In [ ]:
with open('em_bonmin_mean.npy', 'wb') as f:
    np.save(f, f1_em_ecg_2_mean)
with open('em_bonmin_std.npy', 'wb') as f:
    np.save(f, f1_em_ecg_2_std)
with open('bonmin_duration.npy', 'wb') as f:
    np.save(f, duration_em_ecg_2)